In [1]:
import xgboost as xgb

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from numba import jit
import math
import mlflow

import xgboost as xgb

from libs.metrics import smape_fast
from libs.data_engineering import prepareDataXGBoost
import time

In [2]:
size_data = 1000
encoding = "oneHotEncoding"#oneHotEncoding, label
model = 'XGBoost'

In [3]:
train_1 = pd.read_csv('../Data/train_1.csv')
train_1_reduce = train_1

In [4]:
tps1= time.clock()
data = prepareDataXGBoost(train_1_reduce, encoding)
tps2= time.clock()
print("Temps d'exécution total: " + str(tps2-tps1) + " secondes")

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


Temps d'exécution total: 963.9912039999999 secondes


In [5]:
data.head()

Visitors  \
Page                                         Dates                  
!vote_en.wikipedia.org_all-access_all-agents 2015-07-01       3.0   
                                             2015-07-02       4.0   
                                             2015-07-03       7.0   
                                             2015-07-04       4.0   
                                             2015-07-05       4.0   

                                                         Visitors_shift_1  \
Page                                         Dates                          
!vote_en.wikipedia.org_all-access_all-agents 2015-07-01               NaN   
                                             2015-07-02               3.0   
                                             2015-07-03               4.0   
                                             2015-07-04               7.0   
                                             2015-07-05               4.0   

                                                         Visitors_shift_2  \
Page                                         Dates                          
!vote_en.wikipedia.org_all-access_all-agents 2015-07-01               NaN   
                                             2015-07-02               NaN   
                                             2015-07-03               3.0   
                                             2015-07-04               4.0   
                                             2015-07-05               7.0   

                                                         Visitors_shift_3  \
Page                                         Dates                          
!vote_en.wikipedia.org_all-access_all-agents 2015-07-01               NaN   
                                             2015-07-02               NaN   
                                             2015-07-03               NaN   
                                             2015-07-04               3.0   
                                             2015-07-05               4.0   

                                                         Visitors_shift_4  \
Page                                         Dates                          
!vote_en.wikipedia.org_all-access_all-agents 2015-07-01               NaN   
                                             2015-07-02               NaN   
                                             2015-07-03               NaN   
                                             2015-07-04               NaN   
                                             2015-07-05               3.0   

                                                         Visitors_shift_5  \
Page                                         Dates                          
!vote_en.wikipedia.org_all-access_all-agents 2015-07-01               NaN   
                                             2015-07-02               NaN   
                                             2015-07-03               NaN   
                                             2015-07-04               NaN   
                                             2015-07-05               NaN   

                                                         Visitors_shift_6  \
Page                                         Dates                          
!vote_en.wikipedia.org_all-access_all-agents 2015-07-01               NaN   
                                             2015-07-02               NaN   
                                             2015-07-03               NaN   
                                             2015-07-04               NaN   
                                             2015-07-05               NaN   

                                                         Visitors_shift_7  \
Page                                         Dates                          
!vote_en.wikipedia.org_all-access_all-agents 2015-07-01               NaN   
                                             2015-07-02               NaN   
                         

In [6]:
data.shape

(79784650, 23)

In [7]:
data.columns

Index(['Visitors', 'Visitors_shift_1', 'Visitors_shift_2', 'Visitors_shift_3',
       'Visitors_shift_4', 'Visitors_shift_5', 'Visitors_shift_6',
       'Visitors_shift_7', 'Visitors_shift_90', 'agent_all-access_all-agents',
       'agent_all-access_spider', 'agent_desktop_all-agents',
       'agent_mobile-web_all-agents', 'site_commons.wikimedia.org',
       'site_wikipedia.org', 'site_www.mediawiki.org', 'country_de',
       'country_en', 'country_es', 'country_fr', 'country_ja', 'country_ru',
       'country_zh'],
      dtype='object')

In [10]:
data.to_csv('../Data/prepared_data.csv')

<h1>Training</h1>

In [12]:
prepared_data = pd.read_csv('../Data/prepared_data.csv')

In [16]:
prepared_data = prepared_data.set_index(['Page','Dates'])
prepared_data.shape

(79784650, 23)

In [ ]:
metrics_df = pd.DataFrame(columns=['RMSE','SMAPE','MAPE'])

X = prepared_data.drop('Visitors', axis = 1).values
y = prepared_data['Visitors'].values

tscv = TimeSeriesSplit()
print(tscv)

for train_index, test_index in tscv.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    paramSearch = {'max_depth':[2,4,6], 'eta':[1]}
    for max_depth in paramSearch['max_depth']:
        print(max_depth)
        for eta in paramSearch['eta']:
            print(eta)
            with mlflow.start_run():
                num_round = 2
                param = {'max_depth': max_depth, 'eta': eta, 'objective=reg' :'tweedie'}
                bst = xgb.train(param, dtrain, num_round)
                preds = bst.predict(dtest)

                rmse = np.sqrt(mean_squared_error(y_test, preds))
                smape = smape_fast(y_test, preds)
                mape = np.mean(np.abs(preds - y_test)/np.abs(y_test))
                print("Test RMSE: %.3f" % rmse)
                print("Test SMAPE: %.3f" % smape)
                print("Test MAPE: %.3f" % mape)
                new_metrics = pd.DataFrame([(rmse,smape,mape)], columns = ['RMSE','SMAPE','MAPE'])
                metrics_df= metrics_df.append(new_metrics, ignore_index=True)
                
                mean_RMSE = metrics_df['RMSE'].mean()
                sum_RMSE = metrics_df['RMSE'].sum()
                mean_SMAPE = metrics_df['SMAPE'].mean()
                mean_MAPE = metrics_df['MAPE'].mean()
                
                mlflow.log_param("encoding",encoding)
                mlflow.log_param("model",model)
                mlflow.log_param("max_depth",param['max_depth'])
                mlflow.log_param("eta",param['eta'])
                mlflow.log_param("objective",param['objective=reg'])
                mlflow.log_metric('mean_RMSE', mean_RMSE)
                mlflow.log_metric('sum_RMSE', sum_RMSE)
                mlflow.log_metric('mean_SMAPE', mean_SMAPE)
                mlflow.end_run()

TimeSeriesSplit(max_train_size=None, n_splits=5)
TRAIN: [       0        1        2 ... 13297442 13297443 13297444] TEST: [13297445 13297446 13297447 ... 26594883 26594884 26594885]
2
1
[18:25:00] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { objective=reg } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[18:25:00] WARNING: /Users/travis/build/dmlc/xgboost/src/gbm/gbtree.cc:139: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in true_divide


Test RMSE: 6576.314
Test SMAPE: 99.964
Test MAPE: inf
4
1
[18:58:53] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { objective=reg } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[18:58:54] WARNING: /Users/travis/build/dmlc/xgboost/src/gbm/gbtree.cc:139: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in true_divide


Test RMSE: 6551.573
Test SMAPE: 98.248
Test MAPE: inf
6
1
[19:14:11] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { objective=reg } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[19:14:11] WARNING: /Users/travis/build/dmlc/xgboost/src/gbm/gbtree.cc:139: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.


...

Test RMSE: 36826.392
Test SMAPE: 136.273
Test MAPE: inf
4
1
[19:53:21] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { objective=reg } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[19:53:23] WARNING: /Users/travis/build/dmlc/xgboost/src/gbm/gbtree.cc:139: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in true_divide


Test RMSE: 36826.230
Test SMAPE: 136.245
Test MAPE: inf
6
1
[19:58:38] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { objective=reg } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[19:58:39] WARNING: /Users/travis/build/dmlc/xgboost/src/gbm/gbtree.cc:139: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in true_divide


Test RMSE: 36803.761
Test SMAPE: 107.063
Test MAPE: inf
TRAIN: [       0        1        2 ... 53189765 53189766 53189767] TEST: [53189768 53189769 53189770 ... 66487206 66487207 66487208]
